In [1]:
import re
from json import dump
from tqdm import tqdm
import time
from collections import defaultdict
import urllib.request

# user packages
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import pickle

In [4]:
BASE_URL = "https://www.domain.com.au"
N_PAGES = range(1, 5) # update this to your liking

# begin code
url_links = []
property_metadata = defaultdict(dict)

# generate list of urls to visit
for page in N_PAGES:
    url = BASE_URL + f"/rent/melbourne-region-vic/?sort=price-desc&page={page}"
    print(f"Visiting {url}")
    bs_object = BeautifulSoup(urlopen(Request(url, headers={'User-Agent':"PostmanRuntime/7.6.0"})))

    # find the unordered list (ul) elements which are the results, then
    # find all href (a) tags that are from the base_url website.
    index_links = bs_object \
        .find(
            "ul",
            {"data-testid": "results"}
        ) \
        .findAll(
            "a",
            href=re.compile(f"{BASE_URL}/*") # the `*` denotes wildcard any
        )

    for link in index_links:
        # if its a property address, add it to the list
        if 'address' in link['class']:
            url_links.append(link['href'])


Visiting https://www.domain.com.au/rent/melbourne-region-vic/?sort=price-desc&page=1
Visiting https://www.domain.com.au/rent/melbourne-region-vic/?sort=price-desc&page=2
Visiting https://www.domain.com.au/rent/melbourne-region-vic/?sort=price-desc&page=3
Visiting https://www.domain.com.au/rent/melbourne-region-vic/?sort=price-desc&page=4


In [6]:
len(url_links)


80

https://www.domain.com.au/rent/?ssubs=0&postcode=3053

In [7]:
url = BASE_URL+"/rent/?ssubs=0&postcode=3053"

In [8]:
bs_object = BeautifulSoup(urlopen(Request(url, headers={'User-Agent':"PostmanRuntime/7.6.0"})))

In [18]:
int(bs_object.find_all("h1", {"class": "css-ekkwk0"})[0].text.split(" ")[0])

168

In [32]:
count_dict = {'postcode': [], 'count': []}

In [33]:
for postcode in tqdm(range(3000, 3996)):
    url = BASE_URL+f"/rent/?ssubs=0&postcode={postcode}"
    # print(f"Visiting postcode {postcode}", end=" ",flush=True)
    bs_object = BeautifulSoup(urlopen(Request(url, headers={'User-Agent':"PostmanRuntime/7.6.0"})))
    count = int(bs_object.find_all("h1", {"class": "css-ekkwk0"})[0].text.split(" ")[0])
    count_dict['postcode'].append(postcode)
    count_dict['count'].append(count)
    # print(f"Count: {count}", end="\r")
    time.sleep(1)

100%|██████████| 996/996 [24:03<00:00,  1.45s/it]


In [31]:
count_dict["postcode"]

[]

In [27]:
len(count_dict["count"])

117

In [22]:
for i in range(10):
    print(f"Visiting postcode {i}",flush=True)
    time.sleep(1)
    print("Done", end="\r")

Visiting postcode 0
Visiting postcode 1
Visiting postcode 2
Visiting postcode 3


KeyboardInterrupt: 

In [34]:
import json
out_dir = "../data/raw/"
with open(out_dir+"postcode_counts.json", "w") as f:
    json.dump(count_dict, f)

In [1]:
import pandas as pd
import folium
import geopandas as gpd

postcode_zones = gpd.read_file("../data/landing/postcode_zone/POSTCODE_POLYGON.shp").to_crs(epsg=4326)
postcode_count = pd.read_json("../data/raw/postcode_counts.json")

In [ ]:
m = folium.Map(location=[-37.8, 144.97],zoom_start=8)
postcode_zones = gpd.read_file("../data/landing/postcode_zone/POSTCODE_POLYGON.shp").to_crs(epsg=4326)
geoJSON = postcode_zones[['POSTCODE', 'geometry']].drop_duplicates('POSTCODE').to_json()

folium.Choropleth(
    geo_data=geoJSON,   
    name='choropleth',
    data=postcode_count,
    columns=["postcode","count"],
    key_on="properties.POSTCODE",
    fill_color="YlGnBu",
    fill_opacity=0.7,
    line_opacity=0.3,
    legend_name="Count of rental properties"
).add_to(m)
m
# m.save(f"{PLOT_DIR}arev_effi_zone.html")

## scrape

In [4]:
onthehouse_url = "https://www.onthehouse.com.au/property/vic/carlton-3053/1301-6-leicester-st-carlton-vic-3053-18673607"

In [5]:
bs_object = BeautifulSoup(urlopen(Request(onthehouse_url, headers={'User-Agent':"PostmanRuntime/7.6.0"})))

In [17]:
import re
bs_object.findAll(string=re.compile("511,800"))

['Sold on 13 Jun 2024 for $511,800', '$511,800', 'Sold for $511,800']

In [16]:
type(k)

bs4.element.ResultSet

# random code


In [1]:
import requests

payload = { 'api_key': '5b3331933de07d890c95c21bd09a52cd', 'url': 'https://www.oldlistings.com.au/real-estate/VIC/Bentleigh+East/3165/rent/110', 'follow_redirect': 'false' }
r = requests.get('https://api.scraperapi.com/', params=payload)
print(r.text)


<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8" />
<meta name="theme-color" content="#3AB54A" />
<meta http-equiv="x-ua-compatible" content="ie=edge" />
<meta name="viewport" content="width=device-width, initial-scale=1, minimum-scale=1, maximum-scale=1" />
<link rel="next" href="/real-estate/VIC/Bentleigh+East/3165/rent/111" />
<link rel="prev" href="/real-estate/VIC/Bentleigh+East/3165/rent/109" />
<meta name="base.search" content="/real-estate/VIC/Bentleigh+East/3165/rent/" />
<meta name="ROBOTS" content="ALL" />
<meta name="country" content="Australia" />
<meta name="geo.position" content="-37.918170;145.057670" />
<meta name="geo.region" content="VIC" />
<meta name="geo.placename" content="BENTLEIGH EAST" />
<meta name="description" content="Bentleigh East Rental Real Estate Historical Prices Page 110, Old Listings with recorded property information" />
<meta name="keywords" content="Bentleigh East real estate rental listings Page 110" />
<script src="//ajax.googleap

# Preprocess

In [1]:
import bs4
import re
import sys
sys.path.append("../")
from scripts.oldlistings import get_oldlistings_page

test_path = "../data/landing/oldlistings/rent_html/3000_Carlton+South_2.html"

In [2]:
page = bs4.BeautifulSoup(open(test_path,'r').read(), "html.parser")

In [3]:
get_oldlistings_page(page)

[defaultdict(None,
             {'lat': '-37.8033060',
              'lng': '144.9637500',
              'rented_prices': [],
              'address': '603/570 SWANSTON STREET, CARLTON SOUTH',
              'meta_data': [{'label': 'bed',
                'description': 'Bed',
                'quantity': '2'},
               {'label': 'bath', 'description': 'Bath', 'quantity': '1'},
               {'label': 'type',
                'description': 'Category',
                'quantity': 'Apartment'}]}),
 defaultdict(None,
             {'lat': '-37.8042270',
              'lng': '144.9690870',
              'rented_prices': [],
              'address': '63/121 RATHDOWN STREET, CARLTON SOUTH',
              'meta_data': [{'label': 'bed',
                'description': 'Bed',
                'quantity': '2'},
               {'label': 'bath', 'description': 'Bath', 'quantity': '1'},
               {'label': 'car', 'description': 'Car', 'quantity': '1'},
               {'label': 'type',
       